In [ ]:
# August, 2022
# Evgenia Galytska (EG), IUP, UB. 
# Original routine is from Kevin Debeire, DLR. 
# This is the modified version of the F1-score calculations, modified for the Arctic-midlatitude research. 

In [ ]:
import numpy as np
import collections
from sklearn.metrics import f1_score
import sys
sys.path.append('/home/evgenia/tools/Arctic-midlat/Python_scripts/functions')
from collections import defaultdict #
from calculations import get_metric_f1 as f1
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#sns.set(style="darkgrid")


In [ ]:
def function_score_list (dictionary, f1 , reference, reference_p, reference_val): 
    alpha = 0.01

    ref_ds=reference
    score_list = list ()
    ref_p_matrix =  reference_p
    ref_val_matrix =   reference_val 
    for d, key in enumerate (dictionary.keys ()):
    #    print (d, key)
        if d <= len (dictionary.keys ()):
            if key !=ref_ds :
                p_matrix=  dictionary[key]["results"]['p_matrix']
                val_matrix= dictionary[key]["results"]['val_matrix']
                precision, recall, TP, FP, FN, score, auto, count = f1(ref_p_matrix, p_matrix, ref_val_matrix, val_matrix, alpha, 
                        tau_min=0, tau_diff=5, same_sign=True)
                score_list.append([key,score])
    return (score_list)

In [ ]:
def put_row_first(df,row_name):
    #put the row_name row at the first position of the df
    df_out=df
    df_out["new"] = range(1,len(df)+1)
    df_out.loc[row_name, 'new'] = 0
    df_out=df_out.sort_values("new").drop('new', axis=1)
    return df_out
def put_column_first(df,column_name):
    #put the column_name col at the first position of the df
    df_out=df
    col= df_out.pop(column_name)
    df_out.insert(0,column_name,col)
    return df_out


In [ ]:
#Load dictionary with data from Tigramite
dictionary_OND = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_historical_OND10actors_f1.npy",allow_pickle='TRUE').item()
#dictionary_DJF = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_historical_DJF10actors_f1.npy",allow_pickle='TRUE').item()
#dictionary_JFM = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_historical_JFM10actors_f1.npy",allow_pickle='TRUE').item()

#dictionary_scenario_OND = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_scenario_2059-2099_OND10actors_f1.npy",allow_pickle='TRUE').item()
#dictionary_scenario_DJF = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_scenario_2059-2099_DJF10actors_f1.npy",allow_pickle='TRUE').item()
#dictionary_scenario_JFM = np.load("/home/evgenia/tools/Arctic-midlat/output/dicts_for_summary/dict_networks_scenario_2059-2099_JFM10actors_f1.npy",allow_pickle='TRUE').item()


In [ ]:
df_f1score= {}
score_list =[]
alpha = 0.01
#Provide here the dictionary, which should be used (hist/scenario simulations) 
data_source = dictionary_OND
#######
for key1 in data_source:
    for key2 in data_source:
        if not (key1 == key2):
            ref_p_matrix= data_source[key2]['results']['p_matrix']
            p_matrix= data_source[key1]['results']['p_matrix']
            ref_val_matrix= data_source[key2]['results']['val_matrix']
            val_matrix= data_source[key1]['results']['val_matrix']
            precision, recall, TP, FP, FN, score, auto, count = f1(ref_p_matrix, p_matrix, ref_val_matrix, val_matrix, alpha, 
                        tau_min=0, tau_diff=5, same_sign=True)
            score_list.append([key1,key2,score])
key1,key2,score = [list(a) for a in zip(*score_list)]
df_f1score = pd.DataFrame({"model":key1, "ref_model":key2,"F1-score":score})
df_globalavg= df_f1score.groupby(["ref_model", "model"],as_index=False)["F1-score"].mean()
df_globalavg_pivot= df_globalavg.pivot(index="model",columns="ref_model")["F1-score"]
df_globalavg_pivot= put_row_first(df_globalavg_pivot,"OBS")
df_globalavg_pivot= put_column_first(df_globalavg_pivot,"OBS")

In [ ]:
fs = 17
cmap = plt.cm.get_cmap('hot_r')
fig = plt.figure(figsize=(12,9))
g = sns.heatmap(df_globalavg_pivot, cmap=cmap, annot=False, vmin = 0.3, vmax = 0.8, cbar=True)
g.xaxis.set_ticks_position("top")
g.collections[0].colorbar.set_label("F1-score",fontsize=fs)

#g.set_title('Heatmap of pairwise causal network comparison for OND',fontsize=15,weight="bold")
g.set_xlabel('Reference model',fontsize=fs)
g.set_ylabel('Model',fontsize=fs)
g.set_xticklabels(g.get_xticklabels(), rotation="60",verticalalignment= 'baseline',horizontalalignment= "left", fontsize = fs)
g.set_yticklabels(g.get_yticklabels(), fontsize = fs)
#plt.savefig ('/home/evgenia/tools/Arctic-midlat/output/f1score/historical_1979-2019_OND_heatmap.png', dpi =120, bbox_inches = 'tight')